## Recursions and backtracking problems

#### 1) Find the closest pair of points in a 2D-plane

In [1]:
def dist(a, b):
    return (((a[0]-b[0])**2) +((a[1]-b[1])**2))**0.5

In [2]:
def closest_brute_force(points):
    min_dist = float("inf")
    p1 = None
    p2 = None
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            d = dist(points[i], points[j])
            if d < min_dist:
                min_dist = d
                p1 = points[i]
                p2 = points[j]
    return p1, p2, min_dist

In [3]:
def closest(points):
    x_sorted = sorted(points, key=lambda point: point[0])
    y_sorted = sorted(points, key=lambda point: point[1])
    
    def _recursive_find(x, y):
        n = len(x)
        if n<=3:
            return closest_brute_force(x) # base case --> constant time. 
        else:
            midpoint = x[n//2]
            x_left = x[:n//2]
            x_right = x[n//2:]
            y_left = []
            y_right = []
            for point in y: # send left points and right points to their y_left, y_right lists
                y_left.append(point) if (point[0]<=midpoint[0]) else y_left.append(point)
                
            (p1_left, p2_left, d_l) = _recursive_find(x_left, y_left) # search in left half --> T(n/2)
            (p1_right, p2_right, d_r) = _recursive_find(x_right, y_right) # search in right half --> T(n/2)
            
            (p1, p2, d) = (p1_left, p2_left, d_l) if (d_l<d_r) else (p1_right, p2_right, d_r)
            
            # for (a, b) in left and right --> need to search in [-d, d] window around midpoint
            # reducing search space in x direction 
            reduced_in_x = [point for point in y if midpoint[0] - d < point[0] < midpoint[0]+d]
            
            for i in range(len(reduced_in_x)): # search for every point --> O(n)
                # reducing search space in y direction --> need to search only in next 7 points (proof..)
                for j in range(i+1, min(i+7, len(reduced_in_x))): # constant time --> O(1) step
                    d_new = dist(reduced_in_x[i], reduced_in_x[j])
                    if d_new < d:
                        (p1, p2, d) = (reduced_in_x[i], reduced_in_x[j], d_new)
        return p1, p2, d
    
    return _recursive_find(x_sorted, y_sorted)

In [4]:
test_points = [(2,3), (12, 30), (40, 50), (5,1), (12,10), (3,4)]

In [5]:
print(closest(test_points))

((2, 3), (3, 4), 1.4142135623730951)


#### 2)  Phone Number Mnemonics
Given a string digits representing a phone number, return all possible character arrangements that can result from the number.

A numbered telephone keypad looks like so:
- 2 -> "a" || "b" || "c"
- 3 -> "d" || "e" || "f"
- 4 -> "g" || "h" || "i"
- 5 -> "j" || "k" || "l"
- 6 -> "m" || "n" || "o"
- 7 -> "p" || "q" || "r" || "s"
- 8 -> "t" || "u" || "v"
- 9 -> "w" || "x" || "y" || "z"

***Constraints:***
* s will only digits between 2 and 9. 
* 2 <= n <= 10 (constraint on length of string). 

In [6]:
def get_arrangements(digits):
    solution = []
    digit2chars = {'2':'abc', '3':'def', '4':'ghi', '5':'jkl', '6':'mno', '7':'pqrs', '8':'tuv', '9':'wxyz'}
    def _recursive_build(current_str, remaining_digits):
        if len(remaining_digits) == 0:
            solution.append(current_str)
            return 
        for char in digit2chars[remaining_digits[0]]:
            _recursive_build(current_str+char, remaining_digits[1:])
    _recursive_build(current_str='', remaining_digits=digits)
    return solution

In [7]:
Input =  "43"
Output =  ["gd","ge","gf","hd","he","hf","id","ie","if"]

print(get_arrangements(Input))

['gd', 'ge', 'gf', 'hd', 'he', 'hf', 'id', 'ie', 'if']


#### 3) IP Address Restoration

Given a "raw" ip address string s, return all "restored" ip address strings that can be recovered from s.

A "raw" ip address string is a string of digits that can have . marks inserted to create a valid ip address.

***Constraints:*** 
- The raw ip string will only have digits 1 to 9
- 4 <= n <= 12


In [8]:
ip = '255.255.255.255'
org = '255255255255'

In [9]:
ip.split(".")

['255', '255', '255', '255']

In [10]:
def is_valid(ip, original):
    splitted = ip.split(".")
    for section in splitted:
        if len(section) >3:
            return False
        if len(section) < 1:
            return False 
        if int(section)>255:
            return False
    combined = ''.join([section for section in splitted])
    if combined != original:
        return False 
    return True 

In [11]:
is_valid(ip, org)

True

In [12]:
def split_at_idx(string, idx):
    try:
        a = string[0:idx]
    except:
        a = ''
    try: 
        b = string[idx:]
    except:
        b = ''
    return a, b

In [13]:
def restore_ip_address(ip_address):
    solution = set()
    def _recursive_build(processed, remaining, depth):
        if depth == 4:
            if is_valid(processed[1:], ip_address):
                solution.add(processed[1:])
            return
        _recursive_build(processed+'.'+ split_at_idx(remaining, 1)[0], split_at_idx(remaining, 1)[1],depth+1)
        _recursive_build(processed+'.'+ split_at_idx(remaining, 2)[0], split_at_idx(remaining, 2)[1],depth+1)
        _recursive_build(processed+'.'+ split_at_idx(remaining, 3)[0], split_at_idx(remaining, 3)[1],depth+1)
    
    _recursive_build('', ip_address, 0)
    return [item for item in solution]

In [14]:
Input= "255255232132"
Output= ["255.255.232.132"]
    
restore_ip_address(Input)

['255.255.232.132']

In [15]:
Input= "125523213"
Output=[
  "1.255.23.213",
  "1.255.232.13",
  "12.55.23.213",
  "12.55.232.13",
  "125.5.23.213",
  "125.5.232.13",
  "125.52.3.213",
  "125.52.32.13"]

restore_ip_address(Input)

['125.52.3.213',
 '1.255.23.213',
 '1.255.232.13',
 '125.5.23.213',
 '12.55.23.213',
 '125.5.232.13',
 '125.52.32.13',
 '12.55.232.13']

#### 4) Generate The Powerset

Given an input sequence arr, generate its power set.
A "power set" is the set of all subsets that can be formed from a sequence/set.
A set is a collection of distinct objects. A subset is a set that only contains elements found in the original set.

***Constraints:*** 
- All items in the provided sequence will be unique


In [16]:
Input= [1, 2, 3]
Output = [
  [], # the empty set
  [1,2,3],
  [1,2],
  [1,3],
  [1],
  [2,3],
  [2],
  [3]
]

In [17]:
def generate_powerset(items_set):
    solution = []
    def _recursive_build(partial, depth):
        if depth == len(items_set):
            solution.append(partial)
        else:
            _recursive_build(partial+[items_set[depth]], depth+1)
            _recursive_build(partial, depth+1)
    _recursive_build([], depth=0)
    return solution

In [18]:
generate_powerset(Input)

[[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]

#### 5) Palindromic Decompositions

Given a string s, return all of its palindromic decompositions.

A "palindromic decomposition" is a splitting of the string into segments such that each segment is a palindrome.

***Constraints:***
- 0 <= len(s) <= 30

In [19]:
def is_palindrome(string):
    return string == string[::-1]

In [20]:
def get_remaining(remaining, idx, is_string=True):
    try:
        return remaining[idx:]
    except:
        if is_string: 
            return ''
        return []

In [21]:
def get_at_idx(string, idx, as_string=True):
    try:
        return string[idx]
    except:
        if string: 
            return ''
        return []

In [22]:
# Slow, checks palindorme after completion 

def decompose_string(string):
    solution = set()
    splitted = list(string)
    
    def _recursive_build(processed, remaining, depth):
        if depth == len(string):
            for item in processed:
                if not is_palindrome(item):
                    return 
            solution.add(tuple(processed))
        else:
            _recursive_build(processed+[get_at_idx(remaining,0)], get_remaining(remaining, 1), depth+1)
            p_part = get_at_idx(processed, -1)
            if len(p_part) == 0:
                p_part = ''
            r_part = get_at_idx(remaining,0)
            _recursive_build(processed[:-1]+[p_part+r_part], get_remaining(remaining, 1), depth+1)
            
    _recursive_build(processed=[], remaining=string, depth=0)
    return [list(item) for item in solution]

In [23]:
decompose_string("aab")

[['a', 'a', 'b'], ['aa', 'b']]

In [24]:
# improved, checks from the beginning, terminates a path if it sees a non-palindrome early
def partition(string):
    solution = []
    def _recursive_build(processed, depth):
        if depth >= len(string):
            solution.append(processed)
        else:
            for i in range(depth, len(string)):
                if string[depth:i+1] == string[depth:i+1][::-1]: #check if palindrome 
                    _recursive_build(processed+[string[depth:i+1]], i+1)
    _recursive_build([], depth=0)
    return solution

In [25]:
partition('aab')

[['a', 'a', 'b'], ['aa', 'b']]

### 6) Permutations
Given an array arr, return all the permutations of the array.

***Constraints:*** 
- arr will have all unique values. 
- The order you return the permutations does not matter. 

In [26]:
def permute(seq):
    solution = []
    def _recusrive_build(processed, depth):
        if depth == len(seq):
            solution.append(processed)
        else:
            for i in range(depth, len(seq)):
                new_processed = processed.copy()
                new_processed[i], new_processed[depth] = new_processed[depth], new_processed[i]
                _recusrive_build(new_processed, depth+1)
    
    _recusrive_build(seq, depth=0)
    return solution

In [27]:
permute([1,2,3])

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 2, 1], [3, 1, 2]]

In [116]:
def another_permute(seq):
    solution = [] 
    def _recursive_build(processed, remaining, depth):
        if depth == len(seq):
            solution.append(processed)
        else:
            for i in range(len(remaining)):
                new_remaining = remaining.copy()
                _recursive_build(processed + [new_remaining.pop(i)], new_remaining, depth=depth+1)
    _recursive_build(processed=[], remaining=seq, depth=0)
    return solution

In [117]:
another_permute([1,2,3])

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]

### 7) Number of Islands 
A two-dimensional region is divided by a grid into uniform square cells, each of which represents either “land” or “water”.
- The integer 1 is used to represent a square of land.
- The integer 0 is used to represent a square of water.

We define an "island" to be a maximal group of type 1 squares ("land") that are adjacent in one of four directions (horizontally or vertically).

Count the number of distinct islands in the array. 

***Constraints:*** 
- The elements in the input matrix will only be either 0 or 1

In [27]:
# helper_function (flood fill function)

def flood_fill(grid, i, j, new_val):
    n,m = len(grid), len(grid[0])
    old_val = grid[i][j]
    
    if old_val == new_val:
        return
    queue = []
    queue.append((i, j))
    while len(queue) != 0:
        i, j = queue.pop(0)
        if i<0 or i>=n or j<0 or j>= m or grid[i][j] != old_val:
            continue
        else:
            grid[i][j] = new_val
            queue.append((i+1,j))
            queue.append((i-1,j))
            queue.append((i,j+1))
            queue.append((i,j-1))
            
def number_of_islands(grid):
    n, m = len(grid), len(grid[0])
    num_islands = 0
    for i in range(n):
        for j in range(m):
            if grid[i][j] == 1:
                num_islands += 1
                flood_fill(grid, i, j, 'v')
    return num_islands


In [29]:
test_1 = [
 [1,1,1,1,1],
 [1,0,0,0,0],
 [0,0,0,0,1],
 [1,1,0,0,1]
]
# number of islands = 3
print(number_of_islands(test_1))

test_2 = [
 [1,1,1,1,1],
 [0,0,0,0,0],
 [0,0,0,0,0],
 [1,1,1,1,1]
] # number of islands = 2
print(number_of_islands(test_2))

3
2


### 8) Number of Distinct Islands

A two-dimensional region is divided by a grid into uniform square cells, each of which represents either “land” or “water”.
- The integer 1 is used to represent a square of land.
- The integer 0 is used to represent a square of water.

We define an "island" to be a maximal group of type 1 squares ("land") that are adjacent in one of four directions (horizontally or vertically).

Furthermore, we call two islands "distinct" provided that they are unique under translations. That is, one island cannot be shifted horizontally or vertically to obtain the other.

Count the number of distinct islands in the array. 

***Constraints:*** 
- The elements in the input matrix will only be either 0 or 1